## Apply spatial filters to the data 

The reasons are: 
1. Make data smoother.
2. The offline and online fluxes slightly diverge at scales smaller than 50km, so we remove them here. 

In [1]:
import numpy as np
import xarray as xr
#from funcs import *
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors, ticker, cm
from matplotlib.colors import LogNorm
from xgcm import Grid

In [34]:
import sys
sys.path.append('./modules')
import funcs

In [3]:
Model_Tav = xr.open_dataset('./analysis_data_files/ZA_model_Tav_online.nc')
Trac_Tav  = xr.open_dataset('./analysis_data_files/ZA_Trac_Tav_online.nc')

In [4]:
grid = Grid(Model_Tav, periodic='X')

In [5]:
list_trac = ['TRAC01', 'TRAC02', 'TRAC03', 'TRAC04', 'TRAC05', 
            'TRAC06', 'TRAC07', 'TRAC08', 'TRAC09', 'TRAC10', 
            'TRAC11', 'TRAC12', 'TRAC13', 'TRAC14', 'TRAC15', 
            'TRAC16', 'TRAC17', 'TRAC18', 'TRAC19', 'TRAC20']

### Spatial filter 

We need to go from time averages ($\overline{(.)}$) to adding spatial averages ($\left<\overline{(.)}\right>$). 
In particular estimate $\left<\overline{u}\right>$, $\left<\overline{C}\right>$, $\left<\overline{uC}\right>$ etc. 

In [11]:
Model_STav = xr.Dataset()
Lfilter = 100e3 

vars = ['Ttave', 'uVeltave', 'vVeltave', 'wVeltave', 'UTtave', 'VTtave', 'WTtave']

for k in vars:
    Model_STav[k] = funcs.spatial_filter(Model_Tav[k], Lfilter, dims = Model_Tav[k].dims[1:])
# So far all was on the original grid    

In [16]:
# Now we need to make choice of an advection scheme, 
# as we have U and C on different points and we want to estimate a flux. 
# There are many choices here. 

# We make the simple (and easiest choice) to take everything to the center point.
# As this makes life easier at later stages. 
# However, this requires lot more thinking at time of implementation.

Model_STav['UpTp'] = (grid.interp(Model_STav['UTtave'], 'X') - 
                      grid.interp(Model_STav['uVeltave'], 'X')*Model_STav['Ttave'])
Model_STav['VpTp'] = (grid.interp(Model_STav['VTtave'], 'Y', boundary='extend') - 
                      grid.interp(Model_STav['vVeltave'], 'Y', boundary='extend')*Model_STav['Ttave'])
Model_STav['WpTp'] = (grid.interp(Model_STav['WTtave'], 'Z', boundary='extend') - 
                      grid.interp(Model_STav['wVeltave'], 'Z', boundary='extend')*Model_STav['Ttave'])

In [22]:
list_trac = ['TRAC01', 'TRAC02', 'TRAC03', 'TRAC04', 'TRAC05', 
            'TRAC06', 'TRAC07', 'TRAC08', 'TRAC09', 'TRAC10', 
            'TRAC11', 'TRAC12', 'TRAC13', 'TRAC14', 'TRAC15', 
            'TRAC16', 'TRAC17', 'TRAC18', 'TRAC19', 'TRAC20']
list_vels = ['U', 'V', 'W']

Trac_STav = xr.Dataset()

for trac in list_trac: 
    Trac_STav[trac] = funcs.spatial_filter(Trac_Tav[trac], Lfilter, dims = Trac_Tav[trac].dims[1:])
    for vel in list_vels:
        Trac_STav[vel+trac] = funcs.spatial_filter(Trac_Tav[vel+trac], Lfilter, dims=Trac_Tav[vel+trac].dims[1:])

In [26]:
for trac in list_trac:
    Trac_STav['Up'+trac+'p'] = (grid.interp(Trac_STav['U'+trac], 'X') - 
                      grid.interp(Model_STav['uVeltave'], 'X')*Trac_STav[trac])
    Trac_STav['Vp'+trac+'p'] = (grid.interp(Trac_STav['V'+trac], 'Y', boundary='extend') - 
                      grid.interp(Model_STav['vVeltave'], 'Y', boundary='extend')*Trac_STav[trac])
    Trac_STav['Wp'+trac+'p'] = (Trac_STav['W'+trac] - 
                      grid.interp(Model_STav['wVeltave'], 'Z', boundary='extend')*Trac_STav[trac])

In [31]:
Model_STav.load();
Model_STav.to_netcdf('analysis_data_files/ZA_model_STav_online_Lf_100km.nc')

In [32]:
Trac_STav.load();
Trac_STav.to_netcdf('analysis_data_files/ZA_trac_STav_online_Lf_100km.nc')